In [3]:
token = "" # here should be lichess API token, but i will hide my token because its confidential data

In [4]:
import pandas as pd
import numpy as np
import berserk
import json

In [5]:
session = berserk.TokenSession(token)
client = berserk.Client(session=session)

In [6]:
duno = client.games.export_by_player('EvilSaintPL') # posible adding max=300, because without limit it takes some time

In [5]:
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(list(duno), f, ensure_ascii=False, default=str) # saving data without any changes
# some_chess_data = pd.read_json('data.json')
# some_chess_data[:1] 
# im not gonna work on it, instead i will handle data without json.dump() to train working on dicts and list

In [6]:
ranking_data = client.users.get_rating_history('EvilSaintPL')

In [7]:
df1 = pd.read_json(json.dumps(list(duno), default=str))
df = df1.copy()

In [31]:
df = df1.copy()

In [39]:
df.head(1)
# at this point im looing into data, and deciding what data do i need"
# in this case i dont need moves, only a amount of moves played by side
# also its not important if was a game played in tourament
# days per turn have value only for corespondic chess games so i can drop it too
# swiss and initialFen also are not important
# from players -> user i need only one result, so i will take name + will add ranking points to players
# ratingDiff will be calculated later on my own

,id,rated,variant,speed,perf,createdAt,lastMoveAt,status,players,winner,moves,clock,tournament,initialFen,swiss,daysPerTurn
0,uxE9ud30,True,standard,blitz,blitz,2023-06-01 16:22:19.034000+00:00,2023-06-01 16:26:26.523000+00:00,resign,"{'white': {'user': {'name': 'zajc1', 'id': 'za...",black,e4 e5 d3 Nc6 Be3 d5 exd5 Qxd5 Nc3 Bb4 Ne2 Bg4 ...,"{'initial': 300, 'increment': 0, 'totalTime': ...",NaN,NaN,NaN,NaN


# dealing with moves, tournament and players columns

In [32]:
df.moves = df["moves"].str.count(" ")//2 # changing moves column to numer of moves instead of notation
df.drop("tournament", axis=1, inplace=True) # droping unnessesary column
df.drop("initialFen", axis=1, inplace=True)
df.drop("swiss", axis=1, inplace=True)
df.drop("daysPerTurn", axis=1, inplace=True)

In [10]:
df.iloc[0,7]

'mate'

In [33]:
def player(x: dict):
    if ('white' in x.keys()) & ('black' in x.keys()):
        if ("user" in x["white"].keys()) & ("user" in x["black"].keys()):
            plyer_data = {
                "white_player_name" : x['white']['user']['name'],
                "white_ranking" : x['white']['rating'],
                "black_player_name" : x['black']['user']['name'],
                "black_ranking" : x['black']['rating']
            }
        elif ("user" in x["white"].keys()):
            plyer_data = {
                "white_player_name" : x['white']['user']['name'],
                "white_ranking" : x['white']['rating'],
               "black_player_name" : "unknown",
                "black_ranking" : 0
            }
        elif ("user" in x["black"].keys()):
            plyer_data = {
                "black_player_name" : x['black']['user']['name'],
                "black_ranking" : x['black']['rating'],
                "white_player_name" : "unknown",
                "white_ranking" : 0
            }
    else:
        plyer_data = {"white_player_name":"unknown","white_ranking":0,"black_player_name":unknown,"black_ranking":0}
    result = pd.DataFrame(plyer_data, index=[0])
    return result


In [12]:
df.iloc[0,7]

'mate'

In [34]:
c = ["white_player_name","white_ranking","black_player_name","black_ranking"]
finit = df["players"].apply(player)
result=pd.DataFrame(columns=c)
for i in range(len(finit)):
    result = pd.concat([result, finit[i]])
result =result.reset_index().drop("index", axis=1)
df = pd.concat([df, result], axis=1)  # adding separated players columns 
df.drop("players", axis=1, inplace=True) # droping players column

In [39]:
df.head(2)

,id,rated,variant,speed,perf,createdAt,lastMoveAt,status,winner,moves,clock,white_player_name,white_ranking,black_player_name,black_ranking
0,uxE9ud30,True,standard,blitz,blitz,2023-06-01 18:22:19.034000+02:00,2023-06-01 18:26:26.523000+02:00,resign,black,30,300,zajc1,1750,EvilSaintPL,1763
1,DRiFLKVy,True,standard,blitz,blitz,2023-06-01 18:07:38.533000+02:00,2023-06-01 18:17:22.238000+02:00,outoftime,white,69,300,lc1408,1759,EvilSaintPL,1768


# dealing with clock and perf column

In [15]:
df.iloc[0,10]

{'initial': 300, 'increment': 0, 'totalTime': 300}

In [36]:
def clock(x):
    if type(x)==dict:
        if 'initial' in x.keys():
            return x["initial"]
    else:
        return 0

In [35]:
df["clock"]=df["clock"].apply(clock)

{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0, 'totalTime': 300}
{'initial': 300, 'increment': 0

# deling with time columns and simple cleaning

In [38]:
df["createdAt"] = df["createdAt"].apply(lambda x: pd.to_datetime(x).tz_convert('Europe/Berlin'))
df["lastMoveAt"] = df["lastMoveAt"].apply(lambda x: pd.to_datetime(x).tz_convert('Europe/Berlin'))

In [ ]:
df.head(2)

In [61]:
df.head(2)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,white_player_name,white_ranking,black_player_name,black_ranking
0,uxE9ud30,True,standard,blitz,2023-06-01 18:22:19.034000+02:00,2023-06-01 18:26:26.523000+02:00,resign,black,30,300,zajc1,1750,EvilSaintPL,1763
1,DRiFLKVy,True,standard,blitz,2023-06-01 18:07:38.533000+02:00,2023-06-01 18:17:22.238000+02:00,outoftime,white,69,300,lc1408,1759,EvilSaintPL,1768


In [71]:
df.status[df.status=='outoftime'].count()

785

In [43]:
df.drop(df[df.variant!="standard"].index, inplace=True)

In [47]:
df.drop(df[df.speed=="correspondence"].index, inplace=True)
df.drop(df[df.speed=='ultraBullet'].index, inplace=True)

In [50]:
df.drop("perf", axis=1, inplace=True)

In [59]:
df.drop(df[df.status=="noStart"].index, inplace=True)

In [76]:
df.head(2)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,white_player_name,white_ranking,black_player_name,black_ranking
0,uxE9ud30,True,standard,blitz,2023-06-01 18:22:19.034000+02:00,2023-06-01 18:26:26.523000+02:00,resign,black,30,300,zajc1,1750,EvilSaintPL,1763
1,DRiFLKVy,True,standard,blitz,2023-06-01 18:07:38.533000+02:00,2023-06-01 18:17:22.238000+02:00,outoftime,white,69,300,lc1408,1759,EvilSaintPL,1768


In [77]:
df.to_csv('Chess_data_basic.csv')